In [23]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [24]:
#read csv
kidney = pd.read_excel(f"C:\\Users\\lala7\\Desktop\\研究所\\碩一上\\製造數據科學\\HW\\MDS_Assignment02\\MDS_Assignment2_kidney.xlsx", sheet_name='chronic_kidney_disease') 
kidney

,48,80,1.02,1,0,?,normal,notpresent,notpresent.1,121,...,44,7800,5.2,yes,yes.1,no,good,no.1,no.2,ckd
0,7,50,1.02,4,0,?,normal,notpresent,notpresent,?,...,38,6000,?,no,no,no,good,no,no,ckd
1,62,80,1.01,2,3,normal,normal,notpresent,notpresent,423,...,31,7500,?,no,yes,no,poor,no,yes,ckd
2,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
3,51,80,1.01,2,0,normal,normal,notpresent,notpresent,106,...,35,7300,4.6,no,no,no,good,no,no,ckd
4,60,90,1.015,3,0,?,?,notpresent,notpresent,74,...,39,7800,4.4,yes,yes,no,good,yes,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,55,80,1.02,0,0,normal,normal,notpresent,notpresent,140,...,47,6700,4.9,no,no,no,good,no,no,notckd
395,42,70,1.025,0,0,normal,normal,notpresent,notpresent,75,...,54,7800,6.2,no,no,no,good,no,no,notckd
396,12,80,1.02,0,0,normal,normal,notpresent,notpresent,100,...,49,6600,5.4,no,no,no,good,no,no,notckd
397,17,60,1.025,0,0,normal,normal,notpresent,notpresent,114,...,51,7200,5.9,no,no,no,good,no,no,notckd


In [25]:
column_names = ['age', 'blood pressure', 'specific gravity', 'albumin', 'sugar', 'red blood cells', 'pus cell', 'pus cell clumps', 'bacteria', 'blood glucose random', 'blood urea', 'serum creatinine', 'sodium',
       'potassium', 'hemoglobin', 'packed cell volume', 'white blood cell count', 'red blood cell count', 'hypertension', 'diabetes mellitus', 'coronary artery disease', 'appetite', 'pedal edema', 'anemia', 'class']
kidney.columns = column_names

處理並查看各欄缺失值個數

In [26]:
kidney.replace("?", np.nan, inplace=True) # kidney = kidney.replace("?", pd.NA)
kidney = kidney.map(lambda x: np.nan if x == '' else x)

kidney
kidney.isnull().sum().sort_values(ascending=False)

red blood cells            151
red blood cell count       134
white blood cell count     109
potassium                   87
sodium                      86
packed cell volume          72
pus cell                    65
hemoglobin                  52
sugar                       49
specific gravity            47
albumin                     46
blood glucose random        44
blood urea                  19
serum creatinine            17
coronary artery disease     15
appetite                    14
pedal edema                 14
anemia                      14
diabetes mellitus           13
class                       13
blood pressure              12
age                          9
hypertension                 8
bacteria                     4
pus cell clumps              4
dtype: int64

查看各欄變異程度（資訊量）

In [27]:
numerical_col = ['age', 'blood pressure', 'blood glucose random', 'blood urea', 'serum creatinine', 'sodium', 'potassium', 'hemoglobin', 'packed cell volume', 'white blood cell count', 'red blood cell count']
for i in numerical_col:
    print(i)
    print(kidney[i].var(skipna=True))

age
295.5256476171643
blood pressure
187.69463523048296
blood glucose random
6301.2754038354415
blood urea
2556.068860991529
serum creatinine
33.03781267606602
sodium
108.34211927582533
potassium
10.2010238890263
hemoglobin
8.483744565307923
packed cell volume
81.18479953471794
white blood cell count
8728801.932943564
red blood cell count
1.0222361349342481


資料前處理 - 填補遺漏值、轉dummy

In [28]:
# 二元類別變數轉0/1
boolean_mapping_1 = {'yes': 1, 'no': 0}
boolean_mapping_2 = {'normal': 1, 'abnormal': 0}
boolean_mapping_3 = {'present': 1, 'notpresent': 0}
boolean_mapping_4 = {'good': 1, 'poor': 0}
boolean_mapping_5 = {'ckd': 1, 'notckd': 0}

for i in ['hypertension', 'diabetes mellitus', 'coronary artery disease', 'pedal edema', 'anemia']:
    # kidney[i] = kidney[i].replace(boolean_mapping_1)
    kidney[i] = kidney[i].apply(lambda x: str(x).strip().lower() if pd.notna(x) else '')
    kidney[i] = kidney[i].replace(boolean_mapping_1)

for i in ['red blood cells', 'pus cell']:
    kidney[i] = kidney[i].replace(boolean_mapping_2)
for i in ['pus cell clumps', 'bacteria']:
    kidney[i] = kidney[i].replace(boolean_mapping_3)

kidney['appetite'] = kidney['appetite'].replace(boolean_mapping_4)
kidney['class'] = kidney['class'].replace(boolean_mapping_5)

kidney

,age,blood pressure,specific gravity,albumin,sugar,red blood cells,pus cell,pus cell clumps,bacteria,blood glucose random,...,packed cell volume,white blood cell count,red blood cell count,hypertension,diabetes mellitus,coronary artery disease,appetite,pedal edema,anemia,class
0,7.0,50.0,1.020,4.0,0.0,NaN,1.0,0.0,0.0,NaN,...,38.0,6000.0,NaN,0,0,0,1.0,0,0,1.0
1,62.0,80.0,1.010,2.0,3.0,1.0,1.0,0.0,0.0,423.0,...,31.0,7500.0,NaN,0,1,0,0.0,0,1,1.0
2,48.0,70.0,1.005,4.0,0.0,1.0,0.0,1.0,0.0,117.0,...,32.0,6700.0,3.9,1,0,0,0.0,1,1,1.0
3,51.0,80.0,1.010,2.0,0.0,1.0,1.0,0.0,0.0,106.0,...,35.0,7300.0,4.6,0,0,0,1.0,0,0,1.0
4,60.0,90.0,1.015,3.0,0.0,NaN,NaN,0.0,0.0,74.0,...,39.0,7800.0,4.4,1,1,0,1.0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,55.0,80.0,1.020,0.0,0.0,1.0,1.0,0.0,0.0,140.0,...,47.0,6700.0,4.9,0,0,0,1.0,0,0,0.0
395,42.0,70.0,1.025,0.0,0.0,1.0,1.0,0.0,0.0,75.0,...,54.0,7800.0,6.2,0,0,0,1.0,0,0,0.0
396,12.0,80.0,1.020,0.0,0.0,1.0,1.0,0.0,0.0,100.0,...,49.0,6600.0,5.4,0,0,0,1.0,0,0,0.0
397,17.0,60.0,1.025,0.0,0.0,1.0,1.0,0.0,0.0,114.0,...,51.0,7200.0,5.9,0,0,0,1.0,0,0,0.0


In [29]:
# #二元變數get dummies
# binary_col = ['hypertension', 'diabetes mellitus', 'coronary artery disease', 'pedal edema', 'anemia', 'red blood cells', 'pus cell', 'pus cell clumps', 'bacteria', 'appetite', 'class']
# for i in binary_col:
#     dummy = pd.get_dummies(kidney[i], prefix=i, prefix_sep='_', drop_first=True)
#     kidney = pd.concat([kidney, dummy], axis=1)
#     kidney = kidney.drop(columns=i)
# kidney

In [30]:
# #多元變數get dummies
# multi_col = ['specific gravity', 'albumin', 'sugar']
# for i in multi_col:
#     dummy = pd.get_dummies(kidney[i], prefix=i, prefix_sep='_', drop_first=True)
#     kidney = pd.concat([kidney, dummy], axis=1)
#     kidney = kidney.drop(columns=i)
# kidney

#多元變數get dummies
dummy_sg = pd.get_dummies(kidney['specific gravity'], prefix='sp', prefix_sep='_', drop_first=True)
dummy_alb = pd.get_dummies(kidney['albumin'], prefix='alb', prefix_sep='_', drop_first=True)
dummy_sugar = pd.get_dummies(kidney['sugar'], prefix='sugar', prefix_sep='_', drop_first=True)
kidney_input = pd.concat([kidney, dummy_sg, dummy_alb, dummy_sugar], axis=1)
kidney_input = kidney_input.drop(columns=['specific gravity', 'albumin', 'sugar'])
kidney_input.to_csv("kidney_pre.csv")
kidney_input


,age,blood pressure,red blood cells,pus cell,pus cell clumps,bacteria,blood glucose random,blood urea,serum creatinine,sodium,...,alb_1.0,alb_2.0,alb_3.0,alb_4.0,alb_5.0,sugar_1.0,sugar_2.0,sugar_3.0,sugar_4.0,sugar_5.0
0,7.0,50.0,NaN,1.0,0.0,0.0,NaN,18.0,0.8,NaN,...,False,False,False,True,False,False,False,False,False,False
1,62.0,80.0,1.0,1.0,0.0,0.0,423.0,53.0,1.8,NaN,...,False,True,False,False,False,False,False,True,False,False
2,48.0,70.0,1.0,0.0,1.0,0.0,117.0,56.0,3.8,111.0,...,False,False,False,True,False,False,False,False,False,False
3,51.0,80.0,1.0,1.0,0.0,0.0,106.0,26.0,1.4,NaN,...,False,True,False,False,False,False,False,False,False,False
4,60.0,90.0,NaN,NaN,0.0,0.0,74.0,25.0,1.1,142.0,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,55.0,80.0,1.0,1.0,0.0,0.0,140.0,49.0,0.5,150.0,...,False,False,False,False,False,False,False,False,False,False
395,42.0,70.0,1.0,1.0,0.0,0.0,75.0,31.0,1.2,141.0,...,False,False,False,False,False,False,False,False,False,False
396,12.0,80.0,1.0,1.0,0.0,0.0,100.0,26.0,0.6,137.0,...,False,False,False,False,False,False,False,False,False,False
397,17.0,60.0,1.0,1.0,0.0,0.0,114.0,50.0,1.0,135.0,...,False,False,False,False,False,False,False,False,False,False


In [31]:
# 原始資料集填補遺漏值
kidney_input = kidney_input.map(lambda x: np.nan if x == '' else x)
k = 10
imputer = KNNImputer(n_neighbors=k)
imputed_data = imputer.fit_transform(kidney_input)
kidney_imputed = pd.DataFrame(imputed_data, columns=kidney_input.columns.tolist())
kidney_imputed

,age,blood pressure,red blood cells,pus cell,pus cell clumps,bacteria,blood glucose random,blood urea,serum creatinine,sodium,...,alb_1.0,alb_2.0,alb_3.0,alb_4.0,alb_5.0,sugar_1.0,sugar_2.0,sugar_3.0,sugar_4.0,sugar_5.0
0,7.0,50.0,0.6,1.0,0.0,0.0,116.6,18.0,0.8,135.5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,62.0,80.0,1.0,1.0,0.0,0.0,423.0,53.0,1.8,134.7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,48.0,70.0,1.0,0.0,1.0,0.0,117.0,56.0,3.8,111.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,51.0,80.0,1.0,1.0,0.0,0.0,106.0,26.0,1.4,139.3,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,60.0,90.0,0.5,0.7,0.0,0.0,74.0,25.0,1.1,142.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,55.0,80.0,1.0,1.0,0.0,0.0,140.0,49.0,0.5,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
395,42.0,70.0,1.0,1.0,0.0,0.0,75.0,31.0,1.2,141.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396,12.0,80.0,1.0,1.0,0.0,0.0,100.0,26.0,0.6,137.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397,17.0,60.0,1.0,1.0,0.0,0.0,114.0,50.0,1.0,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
#查看相關係數矩陣
correlation_matrix = kidney_imputed.corr()
highly_correlated = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            highly_correlated.append((correlation_matrix.columns[i], correlation_matrix.columns[j]))
for var1, var2 in highly_correlated:
    print(f"{var1}, {var2}:{correlation_matrix.loc[var1, var2].round(2)}")

hemoglobin, packed cell volume:0.87
hemoglobin, red blood cell count:0.72
hemoglobin, class:-0.73
packed cell volume, red blood cell count:0.74


In [33]:
#查看共變異矩陣
df_ckd = kidney_imputed[kidney_imputed['class'] == True]
df_notckd = kidney_imputed[kidney_imputed['class'] == False]

cov_matrix_ckd = df_ckd.cov() 
cov_matrix_notckd = df_notckd.cov()  

print(cov_matrix_ckd)
print(cov_matrix_notckd)


                                 age  blood pressure  red blood cells  \
age                       281.674225       44.638529        -0.119621   
blood pressure             44.638529      232.885286        -0.965411   
red blood cells            -0.119621       -0.965411         0.129026   
pus cell                   -0.298388       -0.608229         0.006187   
pus cell clumps             0.862197       -0.114017         0.016067   
bacteria                    0.035701        0.298640         0.003431   
blood glucose random      265.700405       97.748780        -2.676642   
blood urea                119.539067      100.200200        -0.056572   
serum creatinine            9.127604        8.112209        -0.041127   
sodium                     -5.006246        1.165438         0.053509   
potassium                   4.212930        3.670554         0.135693   
hemoglobin                 -1.680884       -5.245213         0.056059   
packed cell volume        -13.129386      -20.13908

# 切分資料

In [34]:
#切訓練/測試資料
features = kidney_imputed.columns.tolist()
features.remove("class")
X = kidney_imputed[features]
y = kidney_imputed['class'].astype(bool)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train

3       True
18      True
377    False
248     True
177     True
       ...  
71      True
106     True
270    False
348    False
102     True
Name: class, Length: 319, dtype: bool

# 羅吉斯回歸

In [35]:
# 建立羅吉斯回歸模型
logit = LogisticRegression(max_iter=50000)
logit_result = logit.fit(X_train, y_train)

# 在測試資料上進行預測
y_pred = logit.predict(X_test)

# 評估模型表現
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9875


In [36]:
# 建立羅吉斯回歸
X = sm.add_constant(X_train)  
logit = sm.Logit(y_train, X)
logit_result = logit.fit(maxiter=10)
print(logit_result.summary())

         Current function value: 0.000087
         Iterations: 10
                           Logit Regression Results                           
Dep. Variable:                  class   No. Observations:                  319
Model:                          Logit   Df Residuals:                      283
Method:                           MLE   Df Model:                           35
Date:                Wed, 25 Oct 2023   Pseudo R-squ.:                  0.9999
Time:                        21:48:41   Log-Likelihood:              -0.027833
converged:                      False   LL-Null:                       -212.21
Covariance Type:            nonrobust   LLR p-value:                 2.197e-68
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      70.5925    479.303      0.147      0.883    -868.825    1010.010
age                       

C:\Users\lala7\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [37]:
# 記錄各迴歸係數跟它的p-value，並依p-value排序
p_values_df = pd.DataFrame({'Variable': logit_result.pvalues.index, 'P-value': logit_result.pvalues.round(3)})
sorted_p_values_df = p_values_df.sort_values(by='P-value')
print(sorted_p_values_df)

                                        Variable  P-value
appetite                                appetite    0.669
pedal edema                          pedal edema    0.769
hypertension                        hypertension    0.809
packed cell volume            packed cell volume    0.843
potassium                              potassium    0.871
const                                      const    0.883
sp_1.015                                sp_1.015    0.889
sugar_2.0                              sugar_2.0    0.895
coronary artery disease  coronary artery disease    0.900
sp_1.01                                  sp_1.01    0.903
red blood cells                  red blood cells    0.905
alb_1.0                                  alb_1.0    0.916
hemoglobin                            hemoglobin    0.925
sp_1.025                                sp_1.025    0.926
anemia                                    anemia    0.927
sugar_5.0                              sugar_5.0    0.932
sp_1.02       

# 線性判別分析

In [38]:
# 建立線性判别分析模型
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# 在測試資料上進行預測
y_pred = lda.predict(X_test)

# 評估模型表現
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.95


# 二次判別分析模型

In [39]:
# 建立二次判别分析模型
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

# 在測試資料上進行預測
y_pred = qda.predict(X_test)

# 評估模型表現
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9625


C:\Users\lala7\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [40]:
cov_df = pd.concat([X_train, y_train], axis=1)
df_ckd = cov_df[cov_df['class'] == True]
df_notckd = cov_df[cov_df['class'] == False]

cov_matrix_ckd = df_ckd.cov() 
cov_matrix_notckd = df_notckd.cov()  

print(cov_matrix_ckd)


                                 age  blood pressure  red blood cells  \
age                       276.565597       37.178323        -0.235777   
blood pressure             37.178323      250.061328        -0.952440   
red blood cells            -0.235777       -0.952440         0.129852   
pus cell                   -0.434270       -0.480768         0.001006   
pus cell clumps             1.138786        0.126696         0.010688   
bacteria                    0.034790        0.217860        -0.000495   
blood glucose random      257.749604      130.775399        -2.663678   
blood urea                 76.053162       89.596514        -0.133518   
serum creatinine            6.687813        8.693058        -0.009640   
sodium                     -2.758789        3.379470         0.125469   
potassium                   3.821440        4.042268         0.157215   
hemoglobin                 -0.720422       -5.523687         0.039135   
packed cell volume         -8.583529      -20.05626